In [23]:
!pip install pandas rdflib boto3 SPARQLWrapper iribaker

In [24]:
#downlaoding source file name file from S3 local dir
import boto3
s3 = boto3.client('s3')
bucket_name = 'dev-input'
file_key = 'input_raw.csv'
local_file_path = 'tf_recon_mismatch.csv'

s3.download_file(bucket_name, file_key, local_file_path)
print(f"Downloaded {file_key} from S3 bucket {bucket_name}")

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [ ]:
local_file_path = 'tf_recon_mismatch.csv'

In [26]:
import rdflib
from rdflib import Graph as RDFGraph
import csv
from rdflib import Dataset, URIRef, Literal, Namespace, RDF, RDFS, OWL, XSD, BNode, FOAF, SDO
from iribaker import to_iri

In [ ]:
# Create an empty RDF graph
g = rdflib.Graph(bind_namespaces="core")

#create namespace
ex = Namespace("http://example.com/rtf/")

#bind prefix to neamespace
g.bind('', ex)
#g.bind("foaf", FOAF)
g.bind('schema', SDO)

node_subject = 'alm_bsm_contract'

In [ ]:
def parse_and_load_csv_to_rdf(csv_file_path):
    with open(csv_file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        headers = reader.fieldnames
        #print(headers)

        for row in reader:

            #mismatch
            mismatch_field = row['MISMATCHED_FIELD']
            mismatch = URIRef(ex[mismatch_field])        
            
            #split tableName and column name
            mismatch_fields = (mismatch_field.split("."))
            table_name = mismatch_fields[0]
            column_name = mismatch_fields[1]
                       
            #add column type
            column = URIRef(ex[column_name]) 
            g.add((column, RDF.type, ex.column))
            g.add((column, ex['name'], (Literal(column_name, datatype=rdflib.XSD.string))))            
            
            #add table type
            table = URIRef(ex[table_name]) 
            g.add((table, RDF.type, ex.table))
            g.add((table, ex['name'], (Literal(table_name, datatype=rdflib.XSD.string)))) 
            
            #table column to table
            g.add((table, ex.hasColumn, column))
        
            #add table to mismatch
            g.add((mismatch, RDF.type, ex.mismatch))
            g.add((mismatch, ex.hasTable, table))

            property_value = Literal(row['BASELINE_VALUE'], datatype=rdflib.XSD.string)
            g.add((mismatch, ex['baseline_value'], property_value))

            property_value = Literal(row['ENHANCED_VALUE'], datatype=rdflib.XSD.string)
            g.add((mismatch, ex['enhanced_value'], property_value))

            #discrepancy
            compare_key_value = row['COMPARE_KEY_VALUE']
            discrepancy_compare = URIRef(ex[compare_key_value])
            g.add((discrepancy_compare, RDF.type, ex.discrepancy)) 
            g.add((discrepancy_compare, ex['compare_key_value'], (Literal(compare_key_value, datatype=rdflib.XSD.string)))) 

            #add mismatch to discrepancy
            g.add((discrepancy_compare, ex.hasMismatch, mismatch))

            #creating node for Application Run
            run_id = row['RUN_ID']
            application_run = URIRef(ex[run_id])
            g.add((application_run, RDF.type, ex['application_run']))
            g.add((application_run, ex['run_id'], (Literal(run_id, datatype=rdflib.XSD.string))))

            #adding business date property
            property_value = Literal(row['BUSINESS_DATE'], datatype=rdflib.XSD.date)
            g.add((application_run, ex['business_date'], property_value))

            property_value = Literal(row['STRM_IDNN'], datatype=rdflib.XSD.string)
            g.add((application_run, ex['strm_idnn'], property_value))

            #add discrepancy to application
            g.add((application_run, ex.hasDiscrepancy, discrepancy_compare))


In [ ]:
parse_and_load_csv_to_rdf(local_file_path)

In [ ]:
print(g.serialize(format='ttl'))

validation of what loaded in graph

In [196]:
for prefix,ns in g.namespaces():
    print((prefix, ns))

('brick', rdflib.term.URIRef('https://brickschema.org/schema/Brick#'))
('csvw', rdflib.term.URIRef('http://www.w3.org/ns/csvw#'))
('dc', rdflib.term.URIRef('http://purl.org/dc/elements/1.1/'))
('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#'))
('dcmitype', rdflib.term.URIRef('http://purl.org/dc/dcmitype/'))
('dcterms', rdflib.term.URIRef('http://purl.org/dc/terms/'))
('dcam', rdflib.term.URIRef('http://purl.org/dc/dcam/'))
('doap', rdflib.term.URIRef('http://usefulinc.com/ns/doap#'))
('foaf', rdflib.term.URIRef('http://xmlns.com/foaf/0.1/'))
('geo', rdflib.term.URIRef('http://www.opengis.net/ont/geosparql#'))
('odrl', rdflib.term.URIRef('http://www.w3.org/ns/odrl/2/'))
('org', rdflib.term.URIRef('http://www.w3.org/ns/org#'))
('prof', rdflib.term.URIRef('http://www.w3.org/ns/dx/prof/'))
('prov', rdflib.term.URIRef('http://www.w3.org/ns/prov#'))
('qb', rdflib.term.URIRef('http://purl.org/linked-data/cube#'))
('schema', rdflib.term.URIRef('https://schema.org/'))
('sh', rdflib.term

In [ ]:
#Replace the node values.
#g.set(s, p, o)
#g.remove(s,p, o)
#.g.remove(s,none, none)

In [197]:
#Serialize the RDF graph to a file 
output_file = 'output7.ttl'
fmt = 'ttl'
g.serialize(destination=output_file, format=fmt)
print(f"RDF data saved {output_file} successfully.")

RDF data saved output7.ttl successfully.


In [198]:
rdf_data = g.serialize(fromat='ttl')
len(rdf_data)

1523

In [199]:
# Function to execute SPARQL update
def push_triples_to_neptune(sparql_endpoint, query):
    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setMethod(POST)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.addParameter("Accept", "application/sparql-results+json")
    try:
        response = sparql.query().convert()
        print("Data pushed successfully:", response)
    except Exception as e:
        print("Failed to push data:", str(e))

In [200]:
import boto3
from SPARQLWrapper import SPARQLWrapper, POST, JSON
# AWS Neptune configuration
neptune_endpoint = "db-neptune-1.cluster-cfaicgyug7jh.ap-southeast-2.neptune.amazonaws.com"
neptune_port = "8182"  # default port for Neptune
neptune_region = "ap-southeast-2"

# SPARQL endpoint
sparql_endpoint = f"https://{neptune_endpoint}:{neptune_port}/sparql"

In [201]:
# SPARQL Update query to insert data
rdf_data = g.serialize(fromat='ttl')
sparql_insert_query = """
    INSERT DATA { 
            """ + rdf_data + """
        }
"""
#print(sparql_insert_query)

In [202]:
# Push triples to AWS Neptune
push_triples_to_neptune(sparql_endpoint, sparql_insert_query)

Failed to push data: <class 'list'>


In [ ]:
#upload RDF/ttl file to S3 to load by Bulk loader

import boto3
s3 = boto3.client('s3')

bucket_name = 'dev-input'
file_key = 'output.ttl'
output_file_name = 'output.ttl'

s3.upload_file(output_file_name, bucket_name, file_key)
print(f"uploaded {file_key} file to S3 bucket {bucket_name}")

<h3> varifying if data is pushed to Neptune DB </h3>
sometime error shown as Failed to push data. to be invested.

In [203]:
%%sparql
SELECT ?s ?p ?o WHERE {?s ?p ?o} LIMIT 100000

In [124]:
%%sparql
CLEAR ALL